In [37]:
import pandas as pd
import sklearn
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np

from tqdm import tqdm_notebook

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import json

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
def train_test_split(data, train_portion):
    time_len = data.shape[1]
    train_size = int(time_len * train_portion)
    train_data = np.array(data.iloc[:, :train_size])
    test_data = np.array(data.iloc[:, train_size:])
    return train_data, test_data

In [39]:
train_rate = 0.8

In [40]:
trips_data = pd.read_csv(os.path.join('data','trips_time_series_all_inter.csv'),index_col=0)
trips_data= trips_data.fillna(0)

In [41]:
trips_data.head()

,20200221_00,20200221_01,20200221_02,20200221_03,20200221_04,20200221_05,20200221_06,20200221_07,20200221_08,20200221_09,...,20201130_14,20201130_15,20201130_16,20201130_17,20201130_18,20201130_19,20201130_20,20201130_21,20201130_22,20201130_23
01001_AM,291.654,129.650,123.822,280.810,282.418,518.179,736.074,1253.000,981.544,893.510,...,1307.238,1026.083,1073.847,1147.206,650.740,648.024,563.314,735.201,258.461,230.591
01002,192.529,149.992,50.547,73.097,208.114,616.786,732.075,779.587,638.112,734.767,...,701.593,598.760,615.822,826.882,580.843,378.512,381.695,407.566,124.729,106.744
01010_AM,92.726,50.822,37.895,19.586,91.560,396.531,457.180,591.326,516.919,471.008,...,495.460,610.121,559.796,476.880,354.807,215.711,216.593,277.574,119.830,52.331
01031_AM,97.674,64.901,92.150,62.669,109.056,208.462,469.924,691.565,645.793,789.906,...,1085.613,628.526,682.578,582.456,603.958,344.157,286.541,269.655,159.443,118.029
01036,173.565,118.227,204.784,219.318,409.485,779.103,1091.613,969.506,845.084,875.701,...,1070.542,815.612,681.116,746.917,704.784,413.711,441.988,575.437,319.760,68.968


In [42]:
def plot_learning_curves(loss, val_loss):
    plt.plot(np.arange(len(loss)) + 0.5, loss, "b.-", label="Training loss")
    plt.plot(np.arange(len(val_loss)) + 1, val_loss, "r.-", label="Validation loss")
    plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    plt.axis([1, 20, 0, 0.01])
    plt.legend(fontsize=14)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)

### Train/test split

In [43]:
train_data, test_data = train_test_split(trips_data, train_rate)
print("Train data: ", train_data.shape)
print("Test data: ", test_data.shape)

Train data:  (2844, 5452)
Test data:  (2844, 1363)


In [44]:
train_data

array([[291.654, 129.65 , 123.822, ..., 246.314, 219.482, 308.986],
       [192.529, 149.992,  50.547, ..., 155.446,  38.908, 168.601],
       [ 92.726,  50.822,  37.895, ...,  57.156,  58.033,  20.927],
       ...,
       [  0.   ,   0.   ,   0.   , ...,   0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   , ...,  31.704,  12.905,  30.994],
       [  0.   ,   0.   ,   0.   , ...,   0.   ,   0.   ,   0.   ]])

### Scaling

In [45]:
def scale_data(train_data, test_data):
    max_speed = train_data.max()
    min_speed = train_data.min()
    train_scaled = (train_data - min_speed) / (max_speed - min_speed)
    test_scaled = (test_data - min_speed) / (max_speed - min_speed)
    return train_scaled, test_scaled

In [46]:
train_scaled, test_scaled = scale_data(train_data, test_data)

In [47]:
train_scaled.shape

(2844, 5452)

### Sequence data preparation for LSTM

In [48]:
seq_len = 12
pre_len = 1

In [49]:
def sequence_data_preparation(seq_len, pre_len, train_data, test_data):
    trainX, trainY, testX, testY = [], [], [], []

    for i in range(train_data.shape[1] - int(seq_len + pre_len - 1)):
        a = train_data[:, i : i + seq_len + pre_len]
        trainX.append(a[:, :seq_len])
        trainY.append(a[:, -1])

    for i in range(test_data.shape[1] - int(seq_len + pre_len - 1)):
        b = test_data[:, i : i + seq_len + pre_len]
        testX.append(b[:, :seq_len])
        testY.append(b[:, -1])

    
    trainX = np.transpose(np.array(trainX), (1,0,2))
    trainY = np.transpose(np.array(trainY), (1,0))
    testX = np.transpose(np.array(testX), (1,0,2))
    testY = np.transpose(np.array(testY), (1,0))
    
    
    """
    trainX = np.array(trainX)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)
    """

    return trainX, trainY, testX, testY

In [50]:
trainX, trainY, testX, testY = sequence_data_preparation(
    seq_len, pre_len, train_scaled, test_scaled
)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(2844, 5438, 12)
(2844, 5438)
(2844, 1349, 12)
(2844, 1349)


In [51]:
trainX[0][0] * train_data.max()

array([ 291.654,  129.65 ,  123.822,  280.81 ,  282.418,  518.179,
        736.074, 1253.   ,  981.544,  893.51 ,  710.607,  940.438])

## LSTM model

### Model per area

In [52]:
def rescale_data_fn(data_):
    ## Rescale values
    max_speed = train_data.max()

    ## actual train and test values
    data_resc = np.array(data_ * max_speed)
    return data_resc

In [53]:
def train_and_test_lstm_per_area(trainX, trainY, testX, testY):
    test_metrics = []
    for i in tqdm_notebook(range(500)):
        #Reshape for LSTM compliant
        trainX_lite = trainX[i].reshape(trainX[i].shape[0],trainX[i].shape[1],-1)
        trainY_lite= trainY[i].reshape(-1,1)

        testX_lite = testX[i].reshape(testX[i].shape[0],testX[i].shape[1],-1)
        testY_lite= testY[i].reshape(-1,1)

        #print(trainX_lite.shape, trainY_lite.shape, testX_lite.shape, testY_lite.shape)

        model = keras.models.Sequential([
            keras.layers.LSTM(200, return_sequences=True, input_shape=[None, 1]),
            keras.layers.LSTM(200),
            keras.layers.Dense(1)
        ])

        model.compile(loss="mse", optimizer="adam", metrics=['mae', 'mse'])
        history = model.fit(trainX_lite, trainY_lite, epochs=20, verbose=0)#, validation_data=(testX_lite, testY_lite))
        predictY = model.predict(testX_lite)
        #print(testX_lite.shape, predictY.shape)
        testY_lite_rscl = rescale_data_fn(testY_lite)
        predictY_rscl = rescale_data_fn(predictY)
        mae_ = mean_absolute_error(testY_lite_rscl, predictY_rscl)
        mse_ = mean_squared_error(testY_lite_rscl, predictY_rscl)
        rmse_ = mean_squared_error(testY_lite_rscl, predictY_rscl, squared = False)
        print(trips_data.iloc[i].name, mae_, mse_, rmse_)
        test_metrics.append((trips_data.iloc[i].name, mae_, mse_, rmse_))

    return pd.DataFrame.from_records(test_metrics, columns='area_id lstm-MAE lstm-MSE lstm-RMSE'.split())

In [54]:
test_metrics_df= train_and_test_lstm_per_area(trainX, trainY, testX, testY)

01001_AM 239.25063431833547 88946.51132295828 298.2390171036618
01002 147.47978400928875 39803.59297756815 199.50837821396914
01010_AM 127.83951111181443 24787.531891318224 157.44056621886946
01031_AM 149.8006504973234 33978.659704672486 184.33301306242592
01036 203.73167371903992 72795.3015437972 269.8060443055292
01043_AM 153.2755590632547 34156.277486200685 184.81417014450133
01047_AM 125.22453347276385 27107.098663019966 164.6423355732661
01051 77.06697618193517 9785.105384506036 98.91969159124
01058_AM 185.87226630719172 59669.261118335395 244.27292342446674
0105901 517.3210409475826 422770.9584370066 650.2083961600362
0105902 794.0591949505531 1190183.5121521044 1090.9553208780387
0105903 626.9178068730247 672590.9606737149 820.116431169206
0105904 499.1373858174748 385281.62675420847 620.7105821187588
0105905 707.6163327204744 904675.6525315765 951.1443910004288
0105906 579.4297155518528 629951.3001174324 793.6947146840732
01063_AM 152.79941723140635 39412.12451189735 198.524871

03104 580.3851147395784 489346.6637001536 699.5331755536356
03105 89.67652064666862 12734.202894257474 112.84592546590893
03107 148.2277929373049 34236.470150302004 185.03099780929142
03109 108.54903104142174 18865.400363378463 137.3513755423602
03111 194.29264349749812 56579.64759338326 237.8647674486141
03113 274.2297868876758 110062.21779584223 331.7562626324366
03118 156.8515615292288 36691.04229443271 191.54905975867567
03119 399.03939112196826 255922.67933009757 505.88801066055873
03120 131.37190629741647 24277.53086666447 155.81248623478308
03121 501.3207002525018 385866.8091587018 621.1817843101179
0312201 501.37762170871656 385673.6002246755 621.0262476133158
0312202 286.83504916094535 122805.19244587263 350.43571799386064
0312203 485.8715114107288 363562.1344665298 602.961138438067
03123_AM 157.43578011462844 37565.536969033084 193.8183091687498
03128_AM 213.87013690924448 78470.93131491105 280.12663442613064
0313301 405.7049058889922 296240.6691102226 544.2799547201997
03133

06094_AM 128.28473604724564 22694.672344846065 150.6475102510694
06095_AM 158.62018086467768 40899.125451174135 202.2353219671928
06103 138.57637433616915 27508.47665825503 165.85679563483382
06108_AM 143.195584108979 28834.20653569018 169.806379549445
06109_AM 110.01987820771645 20652.869637893957 143.71106303237048
06113_AM 108.03602846337637 17552.241532909145 132.48487284557865
06117_AM 95.92428995029232 14100.934972541152 118.74735774972491
06120_AM 157.36645417232526 35722.37530658718 189.00363834219482
06121_AM 156.91997992689898 34928.78140163824 186.8924327029809
06122 121.68166260068764 23393.123317915986 152.9481066176237
06123_AM 67.93518525540554 7552.513068177361 86.90519586409872
06125_AM 65.69033527448356 8665.262457397326 93.08739150603226
06127_AM 91.60290205465307 13216.22128663237 114.96182534490468
06128_AM 164.66876262427826 41773.348425419455 204.385294053705
06149 155.56286250711244 38309.8258364237 195.72896013728703
06153 369.65968360585293 209396.60489527427 

08069 100.77196567009996 16548.25508542553 128.64002132083752
08072 306.07333441673376 147244.51323236307 383.72452779613013
0807301 433.72274686689224 288108.9481732339 536.7578114692267
0807302 312.44208214388397 157241.76698997087 396.537220182382
0807303 310.55323235183613 141899.91938868398 376.6960570389395
0807304 468.165389978724 327776.73564702214 572.5178911152228
0807305 181.33368477101817 49168.150765541206 221.73892478665357
0807306 363.62823608199363 194502.3213985623 441.02417325874814
08074 258.29671390522424 113621.58764693789 337.07801418505164
08075 172.4015419023541 49761.559531994244 223.07299148932003
08076 439.3909695856706 323215.2081533904 568.5201915089651
08077 606.4181948735354 574040.949696955 757.6549014537918
08085_AM 188.49985770347828 51692.73952979637 227.3603737017433
08086 412.65630354814687 262485.4214479436 512.3333108904237
08088 268.20417674867304 128554.79714116438 358.5453906288078
08089 575.3199940264886 542812.8100246247 736.7583118123777
080

In [ ]:
test_metrics_df.head()

,area_id,lstm-MAE,lstm-MSE,lstm-RMSE
0,01001_AM,239.250634,88946.511323,298.239017
1,01002,147.479784,39803.592978,199.508378
2,01010_AM,127.839511,24787.531891,157.440566
3,01031_AM,149.800650,33978.659705,184.333013
4,01036,203.731674,72795.301544,269.806044


In [ ]:
test_metrics_df.to_csv(os.path.join('data', 'lstm_test_metrics_0_500.csv'))

### Global model

In [ ]:
trainX_all = trainX.reshape(-1,12,1)
trainY_all= trainY.reshape(-1,1)

testX_all = testX.reshape(-1,12,1)
testY_all= testY.reshape(-1,1)

print(trainX_all.shape, trainY_all.shape, testX_all.shape, testY_all.shape)

model = keras.models.Sequential([
    keras.layers.LSTM(200, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(200),
    keras.layers.Dense(10)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(
    trainX_all, 
    trainY_all, 
    epochs=20,
    verbose=0)
     #validation_data=(testX_all, testY_all))

mse_ = model.evaluate(testX_all, testY_all)

print(mse_)

val_mse_per_area['all']= mse_

(15465672, 12, 1) (15465672, 1) (3836556, 12, 1) (3836556, 1)


In [ ]:
with open(os.path.join('data', 'val_mse_per_area_with_all.json'), "w") as outfile:  
    json.dump(val_mse_per_area, outfile) 

In [ ]:
"""
#Reshape for LSTM compliant
trainX_all = trainX[0].reshape(trainX[0].shape[0],trainX[0].shape[1],-1)
trainY_lite= trainY[0].reshape(-1,1)

testX_lite = testX[0].reshape(testX[0].shape[0],testX[0].shape[1],-1)
testY_lite= testY[0].reshape(-1,1)

print(trainX_lite.shape, trainY_lite.shape, testX_lite.shape, testY_lite.shape)

model = keras.models.Sequential([
    keras.layers.LSTM(200, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(200),
    keras.layers.Dense(10)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(trainX_lite, trainY_lite, epochs=20, validation_data=(testX_lite, testY_lite))
"""

In [ ]:
print(
    "Train loss: ",
    history.history["loss"][-1],
    "\nTest loss:",
    history.history["val_loss"][-1],
)

In [ ]:
plot_learning_curves(history.history["loss"], history.history["val_loss"])
plt.show()

In [ ]:
print("That's all folks")